In [6]:
#################################### #frog encryption decryption##############################
import numpy as np
from enum import Enum
import random
import binascii
import random
import time
#import EccCore

BLOCK_SIZE = 16

NUM_ITERATIONS = 8


randomSeed=[
    113, 21,232, 18,113, 92, 63,157,124,193,166,197,126, 56,229,229,
			156,162, 54, 17,230, 89,189, 87,169,  0, 81,204,  8, 70,203,225,
			160, 59,167,189,100,157, 84, 11,  7,130, 29, 51, 32, 45,135,237,
			139, 33, 17,221, 24, 50, 89, 74, 21,205,191,242, 84, 53,  3,230,
			231,118, 15, 15,107,  4, 21, 34,  3,156, 57, 66, 93,255,191,  3,
			85,135,205,200,185,204, 52, 37, 35, 24, 68,185,201, 10,224,234,
			7,120,201,115,216,103, 57,255, 93,110, 42,249, 68, 14, 29, 55,
			128, 84, 37,152,221,137, 39, 11,252, 50,144, 35,178,190, 43,162,
			103,249,109,  8,235, 33,158,111,252,205,169, 54, 10, 20,221,201,
			178,224, 89,184,182, 65,201, 10, 60,  6,191,174, 79, 98, 26,160,
			252, 51, 63, 79,  6,102,123,173, 49,  3,110,233, 90,158,228,210,
			209,237, 30, 95, 28,179,204,220, 72,163, 77,166,192, 98,165, 25,
			145,162, 91,212, 41,230,110,  6,107,187,127, 38, 82, 98, 30, 67,
			225, 80,208,134, 60,250,153, 87,148, 60, 66,165, 72, 29,165, 82,
			211,207,  0,177,206, 13,  6, 14, 92,248, 60,201,132, 95, 35,215,
			118,177,121,180, 27, 83,131, 26, 39, 46, 12]

class ENCRYPTION(Enum):
    ENCRYPT = False
    DECRYPT = True
class FrogIterKey:
    def __init__(self):
        self.xorBu =  [0]*BLOCK_SIZE
        self.SubstPermu = [0]*256
        self.BombPermu = [0]*BLOCK_SIZE

    def size():
        return BLOCK_SIZE*2+256

    def setValue(self,i, value):
        if value < 0:
            value = 256  + value
        if i < BLOCK_SIZE:
            self.xorBu[i] = value
        elif i < BLOCK_SIZE + 256:
            self.SubstPermu[i-BLOCK_SIZE] = value
        else:
            self.BombPermu[i-BLOCK_SIZE-256] = value

    def getValue(self,i):
        if i < BLOCK_SIZE:
            return self.xorBu[i]
        elif i < BLOCK_SIZE + 256:
            return self.SubstPermu[i-BLOCK_SIZE]
        else:
            return self.BombPermu[i-BLOCK_SIZE-256]

    def copyFrom(self,origin):
        for i in range (len(origin.xorBu)):
            self.xorBu[i] = origin.xorBu[i]
        for i in range (len(origin.SubstPermu)):
            self.SubstPermu[i] = origin.SubstPermu[i]
        for i in range (len(origin.BombPermu)):
            self.BombPermu[i] = origin.BombPermu[i]

class FrogInternalKey:
    def __init__(self):
        self.internalKey = [FrogIterKey() for i in range(NUM_ITERATIONS)]
        self.keyE=[FrogIterKey() for i in range(NUM_ITERATIONS)]
        self.keyD=[FrogIterKey() for i in range(NUM_ITERATIONS)]

    def setValue(self,index,value):
        self.internalKey[index/FrogIterKey.size()].setValue(index%FrogIterKey.size(),value)

    def getValue(self,index):
        return self.internalKey[index/FrogIterKey.size()].getValue(index%FrogIterKey.size())



def frogEncrypt(plainText, key):
    #Encrypt plainText using internalKey - (internal cycle) See B.1.1
    for i in range (0, NUM_ITERATIONS):
        for j in range (0, BLOCK_SIZE):
            plainText[j] = plainText[j] ^ key[i].xorBu[j]
            if plainText[j]<0:
                plainText[j] = key[i].SubstPermu[plainText[j]+256]
            else:
                plainText[j] = key[i].SubstPermu[plainText[j]]
            if j< BLOCK_SIZE -1:
                plainText[j+1]= plainText[j+1] ^ plainText[j]
            else:
                plainText[0] = plainText[0] ^ plainText[BLOCK_SIZE-1]
            plainText[key[i].BombPermu[j]] ^= plainText[j]
    return plainText

def frogDecrypt(cipherText, key):
    for i in reversed(range (0, NUM_ITERATIONS)):
        for j in reversed(range (0, BLOCK_SIZE)):
            cipherText[key[i].BombPermu[j]] ^= cipherText[j]
            if(j< BLOCK_SIZE -1):
                cipherText[j+1] = cipherText[j+1] ^ cipherText[j]
            else:
                cipherText[0] = cipherText[0] ^ cipherText[BLOCK_SIZE-1]
            if cipherText[j]<0:
                cipherText[j] = key[i].SubstPermu[cipherText[j]+256]
            else:
                cipherText[j] = key[i].SubstPermu[cipherText[j]]
            cipherText[j] = cipherText[j] ^ key[i].xorBu[j]
    return cipherText

def makeInternalKey(decrypting, keyorigin):
    used = [0]*BLOCK_SIZE
    key= [FrogIterKey() for i in range(NUM_ITERATIONS)]
    k=0
    l=0
    h=0
    for i in range(0, NUM_ITERATIONS):
        key[i]=FrogIterKey()
        key[i].copyFrom(keyorigin[i])
    for i in range (0, NUM_ITERATIONS):
        key[i].SubstPermu=makePermutation(key[i].SubstPermu)
        if(decrypting.value):
            key[i].SubstPermu=invertPermutation(key[i].SubstPermu)

        key[i].BombPermu=makePermutation(key[i].BombPermu)
        for j in range (0, BLOCK_SIZE):
            used[j]=0
        for j in range (0, BLOCK_SIZE-1):
            if(key[i].BombPermu[h] == 0):
                k=h
                while True:
                    k =(k+1) % BLOCK_SIZE
                    if used[k] == 0:
                        break
                key[i].BombPermu[h] = k
                l=k
                while key[i].BombPermu[l] !=k:
                    l=key[i].BombPermu[l]
                key[i].BombPermu[l]=0
            used[h]=1
            h=key[i].BombPermu[h]
        for ind in range (0, BLOCK_SIZE):
            if ind == BLOCK_SIZE -1:
                h=0
            else:
                h=ind+1
            if key[i].BombPermu[ind]==h:
                if(h == BLOCK_SIZE -1):
                    k=0
                else:
                    k=h+1
                key[i].BombPermu[ind]=k

    return key

def hashKey(binaryKey):
    buffer = [0]*BLOCK_SIZE
    simpleKey = [FrogIterKey() for i in range(NUM_ITERATIONS)]
    internalKey= [FrogIterKey() for i in range(NUM_ITERATIONS)]
    for i in range(0, NUM_ITERATIONS):
        simpleKey[i]=FrogIterKey()
        internalKey[i]=FrogIterKey()
    keyLen=len(binaryKey)
    sizeKey=FrogIterKey.size() * NUM_ITERATIONS
    iSeed=0
    iFrase=0
    for i in range(0, sizeKey):
        simpleKey[i//FrogIterKey.size()].setValue(i%FrogIterKey.size(), randomSeed[iSeed]^binaryKey[iFrase])
        if iSeed<250:
            iSeed=iSeed+1
        else:
            iSeed=0
        if iFrase<keyLen-1:
            iFrase=iFrase+1
        else:
            iFrase=0
    simpleKey=makeInternalKey(ENCRYPTION.ENCRYPT, simpleKey)
    for i in range(0, BLOCK_SIZE):
        buffer[i]=0
    last = keyLen-1
    if(last>BLOCK_SIZE):
        last=BLOCK_SIZE-1
    for i in range(0, last+1):
        buffer[i] ^= binaryKey[i]
    buffer[0] ^= keyLen

    position=0

    while True:
        buffer= frogEncrypt(buffer, simpleKey)
        size =sizeKey-position
        if(size> BLOCK_SIZE):
            size=BLOCK_SIZE
        for i in range (0, BLOCK_SIZE):
            if(buffer[i]<0):
                internalKey[(position+i)//FrogIterKey.size()].setValue((position+i)%FrogIterKey.size(), buffer[i]+256)
            else:
                internalKey[(position+i)//FrogIterKey.size()].setValue((position+i)%FrogIterKey.size(), buffer[i])
        position = position + size
        if position == sizeKey:
            break
    return internalKey



def makePermutation( permu):

    #Receives an arbitarty byte arror of (lastElem -1) elements and
    #returns a permutation with values between 0 and lastElem.
	#Reference Text: section B.1.3
    use = [0]*256 # 256 length byte array
    lastElem = len(permu) -1
    last = lastElem
    j=0
    #initialize use array
    for i in range(0, lastElem+1):
        use[i]=i

    for i in range(0, lastElem):
        j = (j+permu[i]) % (last + 1)
        permu[i] = use[j]
        # Remove use[index] value from use array
        if j<last:
            for k in range(j, last):
                use[k] = use[k+1]
        last = last -1
        if j > last:
            j = last
    permu[lastElem] = use[0]
    return permu

def invertPermutation(origPermu):
    #Receives a permutation and returns its inverse
    invPermu = [0]*256
    for i in range(0, len(origPermu)):
        invPermu[origPermu[i]] = i
    return invPermu

def makeKey(k):
    intKey = FrogInternalKey()
    intKey.internalKey = hashKey(k)
    intKey.keyE=makeInternalKey(ENCRYPTION.ENCRYPT, intKey.internalKey)
    intKey.keyD=makeInternalKey(ENCRYPTION.DECRYPT, intKey.internalKey)
    return intKey



###########################################FROG#################################################
#########################################CFB################################################



def cfbEncrypt(body, k, iv):
    """Encrypt data using the CFB mode of operation.

    key is the secret key to use for the symmetric cipher.
    data is the plaintext to encrypt.

    Returns the ciphertext.
    """
    msg=bytes(body, 'utf-8')
    arr = [0]*len(msg)
    encrypted_msg=[""]*len(msg)
    intKey=makeKey(k)
    c_i = frogEncrypt(np.copy(iv), intKey.keyE) # first block is iv
    i = 0
    for ch in msg:
            arr[i] = ch
            i += 1
    i = 0
    while i < len(arr):
        if i + BLOCK_SIZE > len(arr):
            for j in range(0, len(arr) - i):
                encrypted_msg[j+i] = arr[j+i] ^ c_i[j]
            break
        for j in range(0, BLOCK_SIZE):
            encrypted_msg[j+i] = arr[j+i] ^ c_i[j]
        c_i = frogEncrypt(np.copy(encrypted_msg[i:i+BLOCK_SIZE]),intKey.keyE)
        i += BLOCK_SIZE
    return ''.join(chr(i) for i in encrypted_msg)

def cfbDecrypt(body,k,iv):
    bodyList=[ord(c) for c in body]
    pText =[0]*len(bodyList)
    intKey=makeKey(k)
    c_i = frogEncrypt(np.copy(iv), intKey.keyE) # first block is iv
    i = 0
    while i < len(pText):
        if i + BLOCK_SIZE > len(pText):
            for j in range(0, len(pText) - i):
                pText[j+i] = bodyList[j+i] ^ c_i[j]
            break
        for j in range(0, BLOCK_SIZE):
            pText[j+i] = bodyList[j+i] ^ c_i[j]
        c_i = frogEncrypt(np.copy(bodyList[i:i+BLOCK_SIZE]), intKey.keyE)
        i += BLOCK_SIZE
    decrypted_message = ""
    for i in range(len(pText)):
        decrypted_message += chr(pText[i]).encode("utf-8").decode()

    return decrypted_message


#########################################CFB################################################
########################################### EC-ElGamal#############################################


from random import randint
from math import gcd,sqrt
import hashlib

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def find_generator(p):
   factors = []
   n = p - 1
   for i in range(2, int(sqrt(n))+1):
    while n % i == 0:
       factors.append(i)
       n //= i
   if n > 1:
     factors.append(n)

   for g in range(2, p):
     is_generator = True
   for q in factors:
      if pow(g, (p-1)//q, p) == 1:
         is_generator = False
         break
   if is_generator:
       return g


def getKeysElGamal():
    # Use smaller primes for faster testing
    q = randint(10000, 20000)  # Smaller range for testing
    while not is_prime(q):
        q = randint(10000, 20000)

    g = find_generator(q)  # Public value
    while g is None:  # Handle rare cases where no generator is found
        q = randint(10000, 20000)
        while not is_prime(q):
            q = randint(10000, 20000)
        g = find_generator(q)

    x = randint(1, q - 1)  # Private key
    y = pow(g, x, q)  # Public key

    return q, g, x, y


def hash_function(message):
   hash_value = hashlib.sha256(message.encode()).hexdigest()
   return int(hash_value, 16)

def modular_inverse(a, m):
        for x in range(1, m):
            if (((a % m) * (x % m)) % m == 1):
                return x
        return -1

# Signing a message
def sign(m,q,g,x):
   hashMessage=hash_function(m)
   k = randint(1, q-1)
   while gcd(k, q-1) != 1:
     k = randint(1, q-1)
   s1 = pow(g, k, q)

   s2 = ((hashMessage - x*s1) * modular_inverse(k, q-1))% (q-1)

   return s1, s2

# Verifying a signature
def verify(m, s1, s2, y, g, q):
   hashMessage=hash_function(m)
   v1 = pow(g, hashMessage, q)
   v2 = (pow(y, s1, q) * pow(s1, s2, q)) % q
   return v1 == v2








########################################### EC-ElGamal ##################################################
########################################### Rabin Signature #############################################
import hashlib, sys

nrabin = 0x15525796ddab817a3c54c4bea4ef564f090c5909b36818c1c13b9e674cf524aa3387a408f9b63c0d88d11a76471f9f2c3f29c47a637aa60bf5e120d1f5a65221
padding = ''
def gcd(a,b):
    if b > a:
        a,b = b,a
    while b > 0:
        a,b = b,a % b
    return a

def nextPrime(p):
    while p % 4 != 3:
        p = p + 1
    return nextPrime_3(p)

def nextPrime_3(p):
    m_ = 3*5*7*11*13*17*19*23*29
    while gcd(p,m_) != 1:
        p = p + 4
    if (pow(2,p-1,p) != 1):
        return nextPrime_3(p + 4)
    if (pow(3,p-1,p) != 1):
        return nextPrime_3(p + 4)
    if (pow(5,p-1,p) != 1):
        return nextPrime_3(p + 4)
    if (pow(17,p-1,p) != 1):
        return nextPrime_3(p + 4)
    return p

# x: bytes
# return: int
def h(x):
    hx = hashlib.sha256(x).digest()
    idx = len(hx)//2
    hl = hashlib.sha256(hx[:idx]).digest()
    hr = hashlib.sha256(hx[idx:]).digest()
    return int.from_bytes(hl + hr, 'little')

# m: bytes
def root(m, p, q):
    global padding
    i = 0
    while True:
        x = h(m) % nrabin
        sig =   pow(p,q-2,q) * p * pow(x,(q+1)//4,q)
        sig = ( pow(q,p-2,p) * q * pow(x,(p+1)//4,p) + sig ) % (nrabin)
        if (sig * sig) % nrabin == x:
            break
        m = m + bytes.fromhex("00")
        i = i + 1
    #print("paddingnum: " + str(i))
    padding = str(i)
    return sig

def writeNumber(number, fnam):
    with open(fnam + '.txt', 'w') as f:
        f.write('%d' % number)

def readNumber(fnam):
    with open(fnam + '.txt', 'r') as f:
        return int(f.read())

def hF(m, paddingnum):
    return h(m + bytes.fromhex("00") * paddingnum) % nrabin

def sF(hexmsg):
    p = readNumber("p")
    q = readNumber("q")
    return root(bytes.fromhex(hexmsg), p, q)


def vF(hexmsg, paddingnum, s):
    return hF(bytes.fromhex(hexmsg), paddingnum) == (s * s) % nrabin


def generate_key_pair():
    global nrabin
    #print(" generate primes ... ")
   # p = nextPrime( h(bytes.fromhex(sys.argv[2])) % (2**501 + 1) )
   # q = nextPrime( h(bytes.fromhex(sys.argv[2] + '00')) % (2**501 + 1) )
    p = nextPrime( h(bytes.fromhex('01')) % (2**501 + 1) )
    q = nextPrime( h(bytes.fromhex('01' + '00')) % (2**501 + 1) )
    writeNumber(p, 'p')
    writeNumber(q, 'q')
    nrabin = hex(p * q)
    hex_int = int(nrabin, 16)
    nrabin = hex_int
    #print(type(nrabin))
    #print("nrabin = ", hex(p * q))

def create_signature(encrypted_msg):
    sign_public_key = str(encrypted_msg.hex())
    print(("Your digital signature is:\n" + hex(sF(sign_public_key))))
    return hex(sF(sign_public_key)), sign_public_key

def verify_signature(sign_public_key,digital_signature):
    print("The result of your signature verification is: " + str(vF(sign_public_key, int(padding), int(digital_signature, 16))))
    return str(vF(sign_public_key, int(padding), int(digital_signature, 16)))

########################################### Rabin Signature #############################################
def test_cryptographic_components():
    # Test FROG Encryption and Decryption
    print("\n--- FROG Encryption and Decryption Test ---")
    message = "Hello, this test Message!"  # Original message
    print("Original Message:", message)

    # Generate random key and IV
    key = [random.randint(0, 255) for _ in range(BLOCK_SIZE)]
    iv = [random.randint(0, 255) for _ in range(BLOCK_SIZE)]

    # Create FROG encryption keys
    frog_key = makeKey(key)

    # Convert message to bytes
    plaintext_bytes = list(message.encode())
    print("Plaintext Bytes:", plaintext_bytes)

    # Encrypt the plaintext
    ciphertext = frogEncrypt(plaintext_bytes.copy(), frog_key.keyE)
    print("Encrypted Ciphertext Bytes:", ciphertext)

    # Convert the encrypted bytes to a readable format (e.g., Base64)
    encrypted_message = binascii.hexlify(bytearray(ciphertext)).decode()
    print("Encrypted Message (Hex):", encrypted_message)

    # Decrypt the ciphertext
    decrypted_bytes = frogDecrypt(ciphertext.copy(), frog_key.keyD)
    print("Decrypted Bytes:", decrypted_bytes)

    # Convert decrypted bytes back to a string
    try:
        decrypted_message = bytes(decrypted_bytes).decode('utf-8')
        print("Decrypted Message:", decrypted_message)
        assert message == decrypted_message, "Decryption failed!"
    except Exception as e:
        print("Error during decryption or decoding:", e)

    # Test Rabin Signature
    print("\n--- Rabin Signature Test ---")
    generate_key_pair()  # Generate Rabin key pair
    encrypted_msg = bytes(ciphertext)  # Use the ciphertext as the "message" to sign
    digital_signature, public_key = create_signature(encrypted_msg)
    print("Public Key:", public_key)

    # Verify Rabin signature
    verification_result = verify_signature(public_key, digital_signature)
    print("Rabin Signature Verification Result:", verification_result)

if __name__ == "__main__":
    test_cryptographic_components()


--- FROG Encryption and Decryption Test ---
Original Message: Hello, this test Message!
Plaintext Bytes: [72, 101, 108, 108, 111, 44, 32, 116, 104, 105, 115, 32, 116, 101, 115, 116, 32, 77, 101, 115, 115, 97, 103, 101, 33]
Encrypted Ciphertext Bytes: [138, 106, 221, 191, 215, 116, 80, 54, 135, 22, 98, 12, 210, 163, 49, 219, 32, 77, 101, 115, 115, 97, 103, 101, 33]
Encrypted Message (Hex): 8a6addbfd77450368716620cd2a331db204d65737361676521
Decrypted Bytes: [72, 101, 108, 108, 111, 44, 32, 116, 104, 105, 115, 32, 116, 101, 115, 116, 32, 77, 101, 115, 115, 97, 103, 101, 33]
Decrypted Message: Hello, this test Message!

--- Rabin Signature Test ---
Your digital signature is:
0x1f7c024312a12eb8e096ea52ecb3d2765c91a137ffd5cce8c8f62c3698a07b2552d0acbc1e022e0b6e30e7a396b529d5a4c59b303e896c29cfc739e4ef6f03676cc96fc3c68e7d4f65b08ce2d2cf42c4c416d72af8df6bc3212ed697c4f6207562c8967085ffd84057c76a3452b75116993919ab26ba93a34ff7e9fbc4
Public Key: 8a6addbfd77450368716620cd2a331db204d65737361676521
The